<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/FlaskForm_LIFF20210901.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_ngrok
!pip install bert-for-tf2
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 41 kB 148 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=012439ed64e9dcf092ffe4429c4f85c1c22a349081754c57b7dda694a2b60586
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=85eb75983576d0391e33467e8ca40206c5dd159c5e87b3faf9ceda8f8680e830
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=f6e9aed9fa4b6c096316890670a44124571cd729cee321bfaed14dade877218b
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 6.8 MB/s 
     |███████████████████████████

In [2]:
!unzip BERT_20210830_Psychotic_depression_model.zip

Archive:  BERT_20210830_Psychotic_depression_model.zip
   creating: BERT_20210830_Psychotic_depression_model/assets/
  inflating: BERT_20210830_Psychotic_depression_model/keras_metadata.pb  
  inflating: BERT_20210830_Psychotic_depression_model/saved_model.pb  
   creating: BERT_20210830_Psychotic_depression_model/variables/
  inflating: BERT_20210830_Psychotic_depression_model/variables/variables.index  
  inflating: BERT_20210830_Psychotic_depression_model/variables/variables.data-00000-of-00001  


In [ ]:
##AI form (condensed)
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify, render_template
from datetime import datetime
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert 
import os
import numpy
app=Flask(__name__)
run_with_ngrok(app)

BertTokenizer=bert.bert_tokenization.FullTokenizer
bert_layer=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/2',trainable=False)
vocabulary_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=BertTokenizer(vocabulary_file,to_lower_case)
max_len=500
BATCH_SIZE=100
new_model=tf.keras.models.load_model('BERT_20210830_Psychotic_depression_model')

@app.route('/',methods=['POST','GET'])
def index():
  title="輸入病史資料(建議100字以上)"+str(datetime.today())
  myText=request.form.get('myHistory')
  if myText:
    myText_bert=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(str(myText)))
    test_sorted_text_labels=[(myText_bert[:max_len])]
    test_processed=tf.data.Dataset.from_generator(lambda:test_sorted_text_labels,output_types=(tf.int32))
    test_batched=test_processed.padded_batch(BATCH_SIZE,padded_shapes=((max_len,)))
    result=new_model.predict(test_batched)
    myPredict=round(result[0][0],5)
  else:
    myPredict=0
  ##myPredict='50%'
  return render_template("index.html",title=title,myHx=myText,myPredict=myPredict)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://31a4-35-233-193-0.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Sep/2021 14:53:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2021 14:53:27] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2021 14:53:31] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2021 14:53:39] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2021 14:53:47] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2021 14:53:51] "POST / HTTP/1.1" 200 -


In [ ]:
##AI form (too complicate)
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify, render_template
from datetime import datetime
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert 
import os
import numpy
app=Flask(__name__)
run_with_ngrok(app)

class TEXT_Model(tf.keras.Model):
  def __init__(self,vocaulary_size,embedding_dimensions=128,
               cnn_filters=50, dnn_units=512,
               model_output_classes=2, dropout_rate=0.1,
               training=False,name='text_model'):
    super(TEXT_MODEL,self).__init__(name=name)
    self.embedding=layers.Embedding(vocaulary_size,embedding_dimensions)
    self.cnn_layer1=layers.Conv1D(filters=cnn_filters,kernel_size=2,padding='valid',activation='relu')
    self.cnn_layer2=layers.Conv1D(filters=cnn_filters,kernel_size=3,padding='valid',activation='relu')
    self.cnn_layer3=layers.Conv1D(filters=cnn_filters,kernel_size=4,padding='valid',activation='relu')
    self.pool=layers.GlobalMaxPool1D()
    self.dense_1=layers.Dense(units=dnn_units, activation='relu')
    self.dropout=layers.Dropout(rate=dropout_rate)
    self.last_dense=layers.Dense(units=1,activation='sigmoid')
  def call(self,inputs,training):
    l =self.embedding(input)
    l_1=self.cnn_layer1(l)
    l_1=self.poot(l_1)
    l_2=self.cnn_layer2(l)
    l_2=self.pool(l_2)
    l_3=self.cnn_layer3(l)
    l_3=self.pool(l_3)
    concatenated=tf.concat([l_1,l_2,l_3], axis=-1)
    concatenated=self.dense_1(concatnated)
    concatenated=self.dropout(concatenated,training)
    model_output=self.last_dense(concatenated)
    return model_output

  def tokenize_text(text_input):
    return tokenizer.convert_tokens_to_ids(tokenizer.toknize(text_input))

BertTokenizer=bert.bert_tokenization.FullTokenizer
if os.path.isdir('bert_zh_L-12_H-768_A-12_2'):
  bert_layer=hub.KerasLayer('bert_zh_L-12_H-768_A-12_2', trainable=False)
else:
  bert_layer=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/2',trainable=False)
vocabulary_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=BertTokenizer(vocabulary_file,to_lower_case)
max_len=500
BATCH_SIZE=100
new_model=tf.keras.models.load_model('BERT_20210830_Psychotic_depression_model')

@app.route('/',methods=['POST','GET'])
def index():
  title="輸入病史資料(建議100字以上)"+str(datetime.today())
  myText=request.form.get('myHistory')
  if myText:
    #myText_bert=tokenize_text(str(myText))
    myText_bert=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(str(myText)))
    test_sorted_text_labels=[(myText_bert[:max_len])]
    test_processed=tf.data.Dataset.from_generator(lambda:test_sorted_text_labels,output_types=(tf.int32))
    test_batched=test_processed.padded_batch(BATCH_SIZE,padded_shapes=((max_len,)))
    result=new_model.predict(test_batched)
    myPredict=round(result[0][0],5)
  else:
    myPredict=0
  ##myPredict='50%'
  return render_template("index.html",title=title,myHx=myText,myPredict=myPredict)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://03af-35-204-175-255.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Sep/2021 15:18:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2021 15:18:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 15:18:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2021 15:18:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Sep/2021 15:18:33] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2021 15:18:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 15:18:51] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2021 15:18:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 15:20:33] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2021 15:20:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 15:20:55] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2021 15:20:55] "POST / HTTP/1.1" 200 -


In [ ]:
##BASIC no Form
from flask_ngrok import run_with_ngrok
from flask import Flask 
from datetime import date

app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
  return '歡迎使用三總北投門診助手,現在時間是{}'.format(str(date.today()))

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://169f-35-221-198-146.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Sep/2021 07:50:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 07:50:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Sep/2021 07:51:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 07:55:31] "GET / HTTP/1.1" 200 -
